In [ ]:
path='/content/drive/MyDrive/Colab Notebooks/zindi/GeoAI/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
# Load the data
train_data = pd.read_csv(path+'Train.csv')
test_data = pd.read_csv(path+'Test.csv')

In [ ]:
# !pip install pysal
# import geopandas as gpd
# from pysal.lib import weights
# from pysal.model import spreg
# from sklearn.impute import SimpleImputer
# import numpy as np

# # Function to process and impute data
# def impute_spatial_data(data, required_columns, k_neighbors=10):
#     # Ensure all required columns exist in the DataFrame
#     for col in required_columns:
#         if col not in data.columns:
#             data[col] = np.nan

#     # Convert the DataFrame to a GeoDataFrame
#     gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.LON, data.LAT))

#     # Drop rows with missing 'LAT' or 'LON' as spatial weights cannot be calculated without coordinates
#     gdf = gdf.dropna(subset=['LAT', 'LON'])

#     # Create a spatial weights matrix based on k-nearest neighbors
#     knn_weights = weights.KNN.from_dataframe(gdf, k=k_neighbors)

#     # Fill missing values with initial guesses using SimpleImputer (mean strategy)
#     imputer = SimpleImputer(strategy='mean')
#     available_features = [col for col in required_columns if col in gdf.columns]
#     gdf[available_features] = imputer.fit_transform(gdf[available_features])

#     # Spatial Lag Model to handle remaining missing values
#     for col in available_features:
#         if gdf[col].isnull().sum() > 0:
#             # Define the dependent variable (target) and independent variables (features)
#             y = gdf[col].values.reshape(-1, 1)
#             X = gdf.drop(columns=[col, 'ID_Zindi', 'Date', 'ID', 'LAT', 'LON', 'geometry']).values

#             # Fit the Spatial Lag Model
#             model = spreg.ML_Lag(y, X, w=knn_weights, name_y=col, name_x=gdf.columns.drop([col, 'ID_Zindi', 'Date', 'ID', 'LAT', 'LON', 'geometry']).tolist())

#             # Predict missing values
#             predicted_values = model.predy.reshape(-1)

#             # Replace missing values with predicted ones
#             gdf.loc[gdf[col].isnull(), col] = predicted_values[gdf[col].isnull()]

#     # Drop the geometry column and return the imputed data
#     return gdf.drop(columns=['geometry'])

# # Define required columns for train and test data
# train_required_columns = ['LST', 'NO2_trop', 'AAI', 'CloudFraction', 'NO2_strat', 'NO2_total', 'TropopausePressure', 'GT_NO2']
# test_required_columns = ['LST', 'NO2_trop', 'AAI', 'CloudFraction', 'NO2_strat', 'NO2_total', 'TropopausePressure']  # Excludes 'GT_NO2'

# # Process and impute the training data
# train_data = impute_spatial_data(train_data, train_required_columns)

# # Process and impute the test data
# test_data = impute_spatial_data(test_data, test_required_columns)

# # # Save the imputed DataFrames
# # train_data.to_csv(path + 'Train_Imputed.csv', index=False)
# # test_data.to_csv(path + 'Test_Imputed.csv', index=False)

# # Check for any remaining missing values in the datasets
# print("Training data missing values summary:")
# print(train_data.isnull().sum()[train_data.isnull().sum() > 0])

# print("Test data missing values summary:")
# print(test_data.isnull().sum()[test_data.isnull().sum() > 0])


In [ ]:
# # check for missing data and remove if present
# train_data = train_data.dropna()
# train_data.isnull().values.any()

In [ ]:
# # check for missing data and remove if present
# test_data = test_data.dropna()
# test_data.isnull().values.any()

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86584 entries, 0 to 86583
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_Zindi            86584 non-null  object 
 1   Date                86584 non-null  object 
 2   ID                  86584 non-null  object 
 3   LAT                 86584 non-null  float64
 4   LON                 86584 non-null  float64
 5   Precipitation       86584 non-null  float64
 6   LST                 46798 non-null  float64
 7   AAI                 73709 non-null  float64
 8   CloudFraction       73709 non-null  float64
 9   NO2_strat           73709 non-null  float64
 10  NO2_total           73709 non-null  float64
 11  NO2_trop            51111 non-null  float64
 12  TropopausePressure  73709 non-null  float64
 13  GT_NO2              82051 non-null  float64
dtypes: float64(11), object(3)
memory usage: 9.2+ MB


In [ ]:
train_data.head()

,ID_Zindi,Date,ID,LAT,LON,Precipitation,LST,AAI,CloudFraction,NO2_strat,NO2_total,NO2_trop,TropopausePressure,GT_NO2
0,ID_ENTGC7,1/1/19,PD01,45.601585,11.903551,0.000000,NaN,0.230527,0.559117,0.000024,0.000117,NaN,14440.82126,31.0
1,ID_8JCCXC,1/1/19,PD04,45.371005,11.840830,3.047342,NaN,-0.074006,0.869309,0.000024,0.000127,NaN,14441.79815,42.0
2,ID_V3136Z,1/1/19,RO01,45.045825,12.060869,0.000000,NaN,0.024470,0.674160,0.000024,0.000086,NaN,14437.38294,31.0
3,ID_KRVZDJ,1/1/19,RO02,45.104075,11.553241,1.200467,NaN,-0.010442,0.920054,0.000024,0.000124,NaN,14440.83831,30.0
4,ID_PR351A,1/1/19,RO03,45.038758,11.790152,1.274564,NaN,-0.176178,0.747464,0.000024,0.000116,NaN,14438.79037,58.0


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6576 entries, 0 to 6575
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_Zindi            6576 non-null   object 
 1   Date                6576 non-null   object 
 2   ID                  6576 non-null   object 
 3   LAT                 6576 non-null   float64
 4   LON                 6576 non-null   float64
 5   Precipitation       6576 non-null   float64
 6   LST                 3595 non-null   float64
 7   AAI                 5708 non-null   float64
 8   CloudFraction       5708 non-null   float64
 9   NO2_strat           5708 non-null   float64
 10  NO2_total           5708 non-null   float64
 11  NO2_trop            3998 non-null   float64
 12  TropopausePressure  5708 non-null   float64
dtypes: float64(10), object(3)
memory usage: 668.0+ KB


In [ ]:
test_data.head()

,ID_Zindi,Date,ID,LAT,LON,Precipitation,LST,AAI,CloudFraction,NO2_strat,NO2_total,NO2_trop,TropopausePressure
0,ID_2MYNQS,1/1/19,PD03,45.289376,11.642394,3.277529,NaN,-0.313361,0.771456,0.000024,0.000075,NaN,14440.02819
1,ID_P4U5WU,1/1/19,TV03,45.836941,12.510362,0.000000,NaN,-0.229512,0.398208,0.000023,0.000120,NaN,14434.04790
2,ID_U4KWPK,1/1/19,X5561,45.582894,8.842165,0.000000,282.98,-0.470822,0.153694,0.000023,0.000171,0.000148,14427.42478
3,ID_QGSNTZ,1/1/19,X5953,45.131947,10.015742,1.928031,NaN,0.132952,0.756917,0.000024,0.000266,NaN,14443.09006
4,ID_GHSZ6K,1/1/19,X6701,45.186329,9.146666,0.000000,NaN,-0.198272,0.678858,0.000023,0.000149,NaN,14440.85840


In [ ]:
# Repeat the same process for test_data
test_data['Date'] = pd.to_datetime(test_data['Date']) # Correct date format
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86584 entries, 0 to 86583
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_Zindi            86584 non-null  object 
 1   Date                86584 non-null  object 
 2   ID                  86584 non-null  object 
 3   LAT                 86584 non-null  float64
 4   LON                 86584 non-null  float64
 5   Precipitation       86584 non-null  float64
 6   LST                 46798 non-null  float64
 7   AAI                 73709 non-null  float64
 8   CloudFraction       73709 non-null  float64
 9   NO2_strat           73709 non-null  float64
 10  NO2_total           73709 non-null  float64
 11  NO2_trop            51111 non-null  float64
 12  TropopausePressure  73709 non-null  float64
 13  GT_NO2              82051 non-null  float64
dtypes: float64(11), object(3)
memory usage: 9.2+ MB


<ipython-input-28-a7aae58258b2>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test_data['Date'] = pd.to_datetime(test_data['Date']) # Correct date format


In [ ]:
from sklearn.impute import SimpleImputer

# Data preprocessing
# Handling missing values
imputer = SimpleImputer(strategy='mean')  # Keep the imputer

# Impute missing values for all columns with missing data
for col in train_data.columns:
    if train_data[col].isnull().any():  # Check if column has missing values
        train_data[col] = imputer.fit_transform(train_data[[col]])
        # Only impute on columns that exist in test_data
        if col in test_data.columns:
            test_data[col] = imputer.transform(test_data[[col]])

In [ ]:
# Prepare the data for modeling
X_train = train_data.drop(['ID_Zindi', 'Date', 'ID', 'GT_NO2'], axis=1)
y_train = train_data['GT_NO2']
X_test = test_data.drop(['ID_Zindi', 'Date', 'ID'], axis=1)

# Split the dataset for training and validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:

# Model development
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_split, y_train_split)

# Validate the model
predictions = model.predict(X_val_split)
print('RMSE:', np.sqrt(mean_squared_error(y_val_split, predictions)))

# Predict on the test dataset
test_predictions = model.predict(X_test)


RMSE: 9.518925309390882


In [ ]:

# Prepare the submission file
submission = pd.DataFrame({'ID_Zindi': test_data['ID_Zindi'], 'GT_NO2': test_predictions})
submission.to_csv(path+'submission.csv', index=False)
print(submission)


       ID_Zindi     GT_NO2
0     ID_2MYNQS  21.319003
1     ID_P4U5WU  35.124054
2     ID_U4KWPK  37.006351
3     ID_QGSNTZ  31.344253
4     ID_GHSZ6K  37.389201
...         ...        ...
6571  ID_GUSXU9  46.450701
6572  ID_GMVEG1  44.434552
6573  ID_GD6HNP  31.153268
6574  ID_J7YW1Y  43.580250
6575  ID_I4E04N  30.599101

[6576 rows x 2 columns]
